In [11]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
from pathlib import Path
import traceback
from os.path import join
import file_function as ff
from datetime import datetime

In [20]:
# 파일명 수정 결과 즉석 확인
files = ["20453677 김길동 화해계약.PDF",
"20453677 김길동 화해계약서.PDF",
"20453677 김길동 화해(분할).PDF",
"20453677 김길동 화해(분할)계약.PDF",
"20453677 김길동 화해(분할)계약서.PDF",
"20453677 김길동 화해 (분할) 계약서.PDF",
]
#"20414261 1485 장행비 2101000041 550703-2010518 원인서류.pdf",
# "20414336 1631 정효임 2101000389 790917-2114514 원인서류.pdf",
# "20414507 1874 박성관 2101001090 521010-1841917 원인서류.pdf"

# 기본 변수
docu_folder_dict = {"원인서류": "1.원인서류", "양도통지서": "2.양도통지서", "집행권원": "3.집행권원", "강제집행": "4.강제집행", "등본": "5.등초본",
              "초본": "5.등초본", "등초본":"5.등초본", "외국인증명": "5.등초본", "개인회생": "6.개인회생", "신용회복": "7.신용회복", "파산": "8.파산", "재산조사": "9.재산조사", "부채증명서" : "10.부채증명서", "기타": "기타"}

# 필수 요소에 대한 컴파일
p_key = re.compile(r"([\d]{8})[\D]")  # match
docu_kind = r'원인서류|양도통지서[\s]?\d차|양도통지서|집행권원[\s]?[재]?[도]?|강제집행|등초본|(?<!등기부)등본|초본|외국인증명|개인회생|신용회복|파산|재산조사|부채증명서|기타'
p_docu = re.compile(docu_kind)
p_event = re.compile(r"(?<=\D)\d{4}[ㄱ-ㅎ가-힣]{1,3}\d+")

p_basic = re.compile(r'(\d{8})[_\s]?(\D+.+)[_\s]?(' + docu_kind + r')') #event의 스타트 번호 >= basic의 엔드번호
p_out = re.compile('개인회생\(면책\)|파산\(면책\)|환매|매각|종결')

# 파일명 수정을 위한 컴파일--------------------------------
# 날짜
# 년월일 사이에 기호나 공백이 들어가 있는 경우로 찾아서 6자리로 맞춰준다.
# 괄호와 공백을 포함시켜서 모두 제거 대상으로하고 대체 단어에 앞공백만 추가
# 기호를 기준으로 쓰기 때문에 반드시 기호제거보다 나와야
p_day4 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{1})[./\-\s](?P<d>\d{1})(?!\d)\s?\)?')
p_day5 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{1})[./\-\s](?P<d>\d{2})\s?\)?')
p_day6 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{2})[./\-\s](?P<d>\d{2})\s?\)?')
#중간기호는 없고 괄호로 감싸진 경우
p_day_4 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{1})(?!\d)\s?\)+')#괄호로 감싼거
p_day_5 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{2})\s?\)+')#괄호로 감싼거
p_day_6 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1,2})(?P<d>\d{1,2})\s?\)+')#괄호로 감싼거
#괄호,중간기호 없이 숫자만 6/8자리 있는 경우
p_day_d = re.compile(r'(?<!\d)(20)?(?P<y>[0-2][0-9])(?P<m>[0-1][0-9])(?P<d>[0-3][0-9])(?!\d)')
#day변수에 담을 것. 그룹으로 숫자만 day로
p_day = re.compile(r"#(\d{6})")

# 사건번호 - 한글 앞뒤로 띄어쓰기 모두 해버리면 이름 앞뒤로 나오는 숫자까지 포함해버린다. 그리고 연도에 맞게 강화했다.
p_event1=re.compile(r"(?<=\D)([12][09]\d\d)([ㄱ-ㅎ가-힣]{1,3})\s([0-9]+)|(?<=\D)([12][09]\d\d)\s([ㄱ-ㅎ가-힣]{1,3})([0-9]+)")
# event2의 경우 띄어쓰기 안한 거를 event에서 매치를 못하므로 앞뒤 공백 모두 ? 해줘야 겠네...
p_event2=re.compile(r"(?<=\D)(\d{2})\s?([ㄱ-ㅎ가-힣]{1,3})\s?([0-9]+)")
docu_list=["", "원인서류", "양도통지서", "집행권원", "강제집행", "등본", #0~5
"초본", "외국인증명", "개인회생", "신용회복", "파산", "재산조사", "부채증명서", "기타"] 
p1=re.compile(r"원인\s?서류|(입회|가입|카드)\s?신청서|(신용)?\s?대출\s?(신청서)?|(분할)?\s?약정서|녹취록?|통화\s?(내용|내역)|(대출)?\s?원장|마이너스\s?대출")
p2=re.compile(r"(채권)?\s?(양도|양수)\s?통지서?|(채권)?\s?양도\s?및?\s?양수\s?(통지)?서?\s?|(?<![^가-힣][가-힣])양통|(?<=\d차)\s?(양통|양도통지서?)") # 세양통신 해결
p3=re.compile(r"집행\s?권원|승계\s?(집행문)?|판결문|지급\s?명령|이행\s?권고|화해\s?권고") # count=1이 의미있게 하기 위해 올바른 표현도 넣는다.
p4=re.compile(r"강제\s?집행|(채권)?\s?(추심|압류|가압류)?\s?결정(문)?|채권\s?가압류\s?(결정)?(문)?|결정문|(?<!\d)\s?타채|(?<!개시)\s?결정|채권압류\s?및\s?추심명령|배당[가-힣]+") # 결정이라는 말이 여러곳에서 나올 수 있다.ex개인회생 회생결정

p5=re.compile(r'\(?\s?법인\s?등기부\s?(등본)?\s?\)?|\(?\s?(법인)?\s?등본\s?\)?|\(?\s?주민\s?등록\s?등본\s?\)?')
p5except = re.compile(r"\(?\s?등본.{0,3}원?초본\s?\)?|\(?\s?원?초본.{0,3}등본\s?\)?|\(?\s?등\s?초본\s?\)?|\(?\s?주민\s?등록\s?등본\s?및?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?및?\s?등본\s?\)?") # '등본 및 초본' 때문에 .{0,3}
p6=re.compile(r"(?<![가-힣])원초본|(?<=원)원초본|(?<![가-힣])\(?\s?원\s?초본\s?\)?|(?<=[가-힣]{3})원초본|\(?\s?초본\s?\)?|\(?\s?주민\s?등록\s?초본\s?\)?") # 이렇게까지 해야되냐? ㅠㅠ
# p7 외국인증명. 컴파일 구문에서 외국인은 젤 끝에 와야 하는 거 유의
p7 = re.compile(r"\(?\s?외국인\s?증명서?\s?\)?|\(?\s?외국인\s?등록\s?사실\s?증명서?\s?\)?|\(?\s?외국인\s?등록증?\s?\)?|\(?\s?외국인\s?\)?")

# 연도 다음에 나오는 개회가 아닌 경우, 전방탐색을 통해 '개인회생'의 '회생'이 걸리는 거 방지. count도 하자
p8=re.compile(r"(?<!\d)개회|개인\s?회생|(?<!개인)회생")
p9=re.compile(r"(?<=[가-힣]{3})신복|[\s_]신복|신용\s?회복") #이름에있는 신복,숫자뒤 신복은 제외. 이름다음에 띄어쓰기 없이 나온 신복은.. 
p10=re.compile("파산")
p11=re.compile(r"재산\s?조사|재산\s?조회") # 상세문서를 재산조사가 대체하는 게 아님에 유의
p12=re.compile(r"부채\s?증명\s?[서원]?류?")

# 삭제/대체가 아니라 '기타 '를 삽입하는 것이니 search되게끔만 작성하면 됨
p_etc = re.compile("행자부|재산\s?명시|접수증|집행문\s?부여의\s?소|신용\s?조회|배송[가-힣]+|종[적족]\s?조회|화해\s?\(?(분할)?\)?\s?계약|채무\s?조정|(채무감면|분할\s?상환)")

# 제거할거. 중간에 있는 숫자 하나는 양통때메 애매하네..
#p_rm = re.compile(r"\(?\s?[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}\s?\)?|\(?\s?[a-zA-Z0-9]{4,}-[a-zA-Z0-9]{4,}\s?\)?|\(?\s?\d{10,}\s?\)?|(?<![가-힣\d])[3-9]\d{5,}(?!가-힣\d])|(?<![가-힣\d])\d{4}(?![가-힣\d])")
p_rm = re.compile(r"TAA\(회\)|SCSB|[a-zA-Z\-\d_]{2,}_[a-zA-Z\-\d_]{4,}|\(?\s?[a-zA-Z][\d]{9,}\s?\)?|\(?\s?[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}\s?\)?|\(?\s?[a-zA-Z0-9]{4,}-[a-zA-Z0-9]{4,}\s?\)?|\(?\s?\d{10,}\s?\)?|(?<![가-힣\d])[13-9]\d{5,}(?!가-힣\d])|(?<![가-힣\d])\d{3,4}(?![가-힣\d])|(?<![가-힣\d])\d{6}[\-_]\d{6}(?!가-힣\d])|(-)?\d{1,3}(,\d{3})+(\.\d+)?")
#                                 언더바로 연결된 영어,숫자 키(C1_3284792)                         2개의 하이픈으로 연결된 영어 및 숫자          하이픈 또는 언더바로 연결된 4자리 이상의 영숫자(주민번호 포함)  10자리키          날짜 아닐 6자리 이상 단독 숫자      3~4자리의 단독 숫자                 앞과 교집합있지만, 확실한 주민번호       금액
p_sign = re.compile("[^a-zA-Zㄱ-ㅎ가-힣0-9\s_()]|\(\s?\)|_ | _|  ")

# -----------------------------------------------------------

for f in tqdm(files):
    print("==============")

    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]
    print(f"76 stem_n = {n}, ext = {ext}") # 중지점======================================================
    depth1, depth2, depth3,  = "", "", ""
    key, name, docu, event, extra, day = "", "", "", "", "", ""

    # key 없는 거 파일명에러 폴더로
    if not p_key.match(n):  
        print(f, "noKey")
        continue
    else : 
        key = p_key.match(n).group(1)##############################  key

    # 연속공백 처리1
    n = re.sub("[\s]{2,}", " ", n)

    print(f"89 공백 후 n = {n}, key = {key}") # 중지점======================================================
    # 날짜, 주민번호 내부 공백 제거 및 예외 log추가-------
    if p_day4.search(n):
        n = p_day4.sub(r"#\g<y>0\g<m>0\g<d>", n)
    elif p_day5.search(n) :
        n = p_day5.sub(r"#\g<y>0\g<m>\g<d>", n)
    elif p_day6.search(n) :
        n = p_day6.sub(r"#\g<y>\g<m>\g<d>", n)
    elif p_day_4.search(n) :
        n = p_day_4.sub(r"#\g<y>0\g<m>0\g<d>", n)
    elif p_day_5.search(n) :
        n = p_day_5.sub(r"#\g<y>0\g<m>\g<d>", n)
    elif p_day_6.search(n) :
        n = p_day_6.sub(r"#\g<y>\g<m>\g<d>", n)
    elif p_day_d.search(n[8:]) :
        n = n[:8] + p_day_d.sub(r"#\g<y>\g<m>\g<d>", n[8:])
    else : pass

    if p_day.search(n) :
        day = p_day.search(n).group(1) #제거
        n = p_day.sub(" ", n)

    print(f"110 n = {n}") # 중지점======================================================
    # 사건번호 : 내부 공백 제거 및 연도에 20 없으면 추가
    if p_event1.search(n):
        print("e1")
        n=p_event1.sub(r'\1\2\3', n)
        # 1) + 사건번호 연도가 yy라면
    elif p_event2.search(n):
        print("e2")
        n=p_event2.sub(r'20\1\2\3', n)
    else : pass 

    # 법원문서 중 문서구분 서치 범위 설정
    e_s = None
    if p_event.search(n) :
        e_s = p_event.search(n).start()

    print(f"128 n = {n}") # 중지점======================================================
    # 타기관 키값 : 날짜, 사건번호 뒤 기호 앞
    n = n[:8] + p_rm.sub(" ", n[8:]) 
    # 기호정리, 연속공백 : set임 순서 주의
    n = p_sign.sub(" ", n)
    n = re.sub("[\s]{2,}", " ", n)

    print(f"135 n = {n}") # 중지점======================================================

    # 문서구분 수정하기 -------------------------------
    # 기타가 들어가면 if-elif에 따라 elif를 건너뛰게 되고, 기타추가하기만 실행된다.
    # 기타가 들어있되(!none), 부채증명서가 들어있지 않은(none) 경우만 기타 추가하기로~
    if (re.search("기타", n)!=None) and (p12.search(n)==None) : pass
    # 원인서류
    elif p1.search(n):
        n=p1.sub(docu_list[1], n, count=1)
        print("p1")
    # 양도통지서
    elif p2.search(n):
        n=p2.sub(docu_list[2], n, count=1)  # 이름에 양통들어가는 경우가 있을 수 있어서 공백씀..
        print("p2")
    # 등초본
    elif p5except.search(n) :
        n=p5except.sub("등초본", n, count=1)
        print("p5except")
    # 등본(법인등기부등분)
    elif p5.search(n) :
        n=p5.sub(docu_list[5], n, count=1)   
        print("p5")
    # 초본
    elif p6.search(n):
        n=p6.sub(docu_list[6], n, count=1)
        print("p6")
    # 외국인 증명
    elif p7.search(n):
        n=p7.sub(docu_list[7], n, count=1)
        print("p7")
    
    #파산 강제집행의 배당과 구별하기 위해 왔음. 파산은 문서구분에 웬만하면 들어가니 먼저 걸러주기
    elif p10.search(n[:e_s]) : pass
    # 집행권원
    elif p3.search(n[:e_s]):
        n=p3.sub(docu_list[3], n, count=1)
        print("p3")
    # 강제집행
    elif p4.search(n[:e_s]) :
        n=p4.sub(docu_list[4], n, count=1)
        print("p4")
    #개인회생
    elif p8.search(n[:e_s]):
        n=p8.sub(docu_list[8], n, count=1) # 개회 반복될 수 있음
        print("p8")
    #신용회복
    elif p9.search(n[:e_s]):
        n=p9.sub(docu_list[9], n, count=1)
        print("p9")
    #재산조사
    elif p11.search(n):
        n=p11.sub(docu_list[11], n, count=1)
        print("p11")
    #부채증명서
    elif p12.search(n):
        n=p12.sub(docu_list[12], n, count=1)
        n=re.sub("기타", "", n)
        print("p12")

    #위와 별개임에 유의-----------------
    #기타 추가하기
    if (p_docu.search(n)==None) and p_etc.search(n) :
        etc = p_etc.search(n)
        rep = "기타 " + etc.group()
        n = p_etc.sub(rep, n)

    

# 새 파일명(new_f) 만들기__________________________________________________________
    if p_docu.search(n):
        d_obj = p_docu.search(n)##############################  docu, name
        name = n[8:d_obj.start()].replace("_", " ").strip() #replace가 먼저나와야
        docu = d_obj.group().strip() #양통, 집행권원 때문에 strip 필요
    else :
        print(f, ">>>", n, "다큐없음")
        print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")
        continue

    
    
    if not p_event.search(n) : #사건번호 없으면
        ##############################  extra
        extra=n[d_obj.end():].strip().replace(" ", "_") #strip이 먼저 나와야

    else :  # 사건번호 있으면
        e_obj = p_event.search(n)
        event = e_obj.group()##############################  event
        extra = n[d_obj.end():e_obj.start()].strip().replace(" ", "_") #strip이 먼저 나와야
        extra = extra + "_" +n[e_obj.end():].strip().replace(" ", "_")##############################  extra
        if d_obj.end() > e_obj.start() : # 사건번호앞에 문서구분 없는 경우
            print(f,">>>", n+ext, "문서가 사건 뒤에")
            print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")
            continue
        else : pass #문서구분 - 사건번호 


##################################################################################################
    if not p_docu.search(n) : #docu없으면
        print([f, "nodocu"])
        print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")
        continue

    else : #docu 있으면
        d_obj = p_docu.search(n)##############################  docu, name
        name = n[8:d_obj.start()].replace("_", " ").strip() #replace가 먼저나와야
        docu = d_obj.group().strip() #양통, 집행권원 때문에 strip 필요

        if len(name) < 2 : 
            print(f"name 이상")
            print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")
            continue
        else : pass
        
        
        if not p_event.search(n) : #사건번호 없으면
            ##############################  extra
            extra=n[d_obj.end():].strip().replace(" ", "_") #strip이 먼저 나와야
        else :  # 사건번호 있으면
            e_obj = p_event.search(n)
            event = e_obj.group()##############################  event
            extra = n[d_obj.end():e_obj.start()].strip().replace(" ", "_") #strip이 먼저 나와야
            extra = extra + "_" +n[e_obj.end():].strip().replace(" ", "_")##############################  extra
            if d_obj.end() > e_obj.start() : # 사건번호앞에 문서구분 없는 경우
                print([f, "nodocu"])
                print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")
                continue
            else : pass #문서구분 - 사건번호 

    # new_f
    # continue 모두 해줬기 때문에 try 안에만 있으면 된다(현재는 key있는 경우)
    name_items = [key, name, docu]
    if event : name_items.append(event)
    if extra : name_items.append(extra) #일련번호나 _로 끝나는 경우는 re_name에서 제거해줌
    if day : name_items.append(day)

    new_f = "_".join(name_items)+ext
    # 마지막에 _ 두개 인 경우 꼭 해줘야 해.
    new_f = re.sub("[_]{2,}", "_", new_f)

    print(f, ">>>", new_f)
    print(f"key:{key}, name:{name}, docu:{docu}, event:{event}, extra:{extra}, day:{day}")


##############################################################################################            
    # 이름
    depth1 = ""
    depth3=key
    
    # depth1
    for key, value in docu_folder_dict.items():
        if re.search(key, docu):
            depth1=value
            print(f"depth1 :{depth1}")
            break
   

  0%|          | 0/6 [00:00<?, ?it/s]

76 stem_n = 20453677 김길동 화해계약, ext = .PDF
89 공백 후 n = 20453677 김길동 화해계약, key = 20453677
110 n = 20453677 김길동 화해계약
128 n = 20453677 김길동 화해계약
135 n = 20453677 김길동 화해계약
20453677 김길동 화해계약.PDF >>> 20453677_김길동_기타_화해계약.PDF
key:20453677, name:김길동, docu:기타, event:, extra:화해계약, day:
depth1 :기타
76 stem_n = 20453677 김길동 화해계약서, ext = .PDF
89 공백 후 n = 20453677 김길동 화해계약서, key = 20453677
110 n = 20453677 김길동 화해계약서
128 n = 20453677 김길동 화해계약서
135 n = 20453677 김길동 화해계약서
20453677 김길동 화해계약서.PDF >>> 20453677_김길동_기타_화해계약서.PDF
key:20453677, name:김길동, docu:기타, event:, extra:화해계약서, day:
depth1 :기타
76 stem_n = 20453677 김길동 화해(분할), ext = .PDF
89 공백 후 n = 20453677 김길동 화해(분할), key = 20453677
110 n = 20453677 김길동 화해(분할)
128 n = 20453677 김길동 화해(분할)
135 n = 20453677 김길동 화해(분할)
20453677 김길동 화해(분할).PDF >>> 20453677 김길동 화해(분할) 다큐없음
key:20453677, name:, docu:, event:, extra:, day:
76 stem_n = 20453677 김길동 화해(분할)계약, ext = .PDF
89 공백 후 n = 20453677 김길동 화해(분할)계약, key = 20453677
110 n = 20453677 김길동 화해(분할)계약
128 n = 20453677

In [49]:
files = ["20415347 220406 정상길 6001001674 80,02,06-1101016 (220406 )원인서류.pdf",
"20416721 220406 장영숙 6001004357 620207-2111019 .pdf",
"20416721 220406 장영숙 6001004357 620207-2111019 원인서류.pdf",
"20416721 220406 장영숙 6001004357 620207-2111019.pdf",
]
p_day4 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{1})[./\-\s](?P<d>\d{1})(?!\d)\s?\)?')
p_day5 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{1})[./\-\s](?P<d>\d{2})\s?\)?')
p_day6 = re.compile(r'\s?\(?\s?(?<=\D)(20|19)?(?P<y>\d{2})[./\-\s](?P<m>\d{2})[./\-\s](?P<d>\d{2})\s?\)?')
#중간기호는 없고 괄호로 감싸진 경우
p_day_4 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{1})(?!\d)\s?\)+')#괄호로 감싼거
p_day_5 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1})(?P<d>\d{2})\s?\)+')#괄호로 감싼거
p_day_6 = re.compile(r'\s?\(+\s?(20|19)?(?P<y>\d{2})(?P<m>\d{1,2})(?P<d>\d{1,2})\s?\)+')#괄호로 감싼거
#숫자만 6/8자리 있는 경우
p_day_d = re.compile(r'(?<!\d)(20)?(?P<y>[0-2][0-9])(?P<m>[0-1][0-9])(?P<d>[0-3][0-9])(?!\d)')
#day변수에 담을 것
p_day = re.compile(r"#(\d{6})")


for f in files :
    n = os.path.splitext(f)[0]
    n = re.sub("[\s]{2,}", " ", n)
    
    # 날짜, 주민번호 내부 공백 제거 및 예외 log추가-------
    if p_day4.search(n):
        n = p_day4.sub(r"#\g<y>0\g<m>0\g<d>", n)
        print("1")
    elif p_day5.search(n) :
        n = p_day5.sub(r"#\g<y>0\g<m>\g<d>", n)
        print("2")
    elif p_day6.search(n) :
        n = p_day6.sub(r"#\g<y>\g<m>\g<d>", n)
        print("3")
    elif p_day_4.search(n) :
        n = p_day_4.sub(r"#\g<y>0\g<m>0\g<d>", n)
        print("4")
    elif p_day_5.search(n) :
        n = p_day_5.sub(r"#\g<y>0\g<m>\g<d>", n)
        print("5")
    elif p_day_6.search(n) :
        n = p_day_6.sub(r"#\g<y>\g<m>\g<d>", n)
        print("6")
    elif p_day_d.search(n[8:]) :
        print("7")
        n = n[:8] + p_day_d.sub(r"#\g<y>\g<m>\g<d>", n[8:])
    else : pass

    if p_day.search(n) :
        print("day")
        day = p_day.search(n).group(1) #제거
        n = p_day.sub(" ", n)
        print("day : ", day)

    print(f + " >>> " + n)
    print("-----------------------------")

6
day
day :  220406
20415347 220406 정상길 6001001674 800206-1101016 (220406 )원인서류.pdf >>> 20415347 220406 정상길 6001001674 800206-1101016 원인서류
-----------------------------
7
day
day :  220406
20416721 220406 장영숙 6001004357 620207-2111019 .pdf >>> 20416721   장영숙 6001004357 620207-2111019 
-----------------------------
7
day
day :  220406
20416721 220406 장영숙 6001004357 620207-2111019 원인서류.pdf >>> 20416721   장영숙 6001004357 620207-2111019 원인서류
-----------------------------
7
day
day :  220406
20416721 220406 장영숙 6001004357 620207-2111019.pdf >>> 20416721   장영숙 6001004357 620207-2111019
-----------------------------


In [10]:
files = ["20415347 3073 정상길 601,001,674 800206-1101016 원인서류 1차.pdf",
"20414261 1485 장행비 2,101,000.041_550703-2010518 원인서류 2차.pdf",
"20414336 1631 정효임 2,101.000389 790917-2114514 원인서류_3.pdf",
"20414507 1874 박성관 2101001090 521010-1841917 원인서류_4.pdf",
]
p_rm = re.compile(r"\(?\s?(?<!\d)\d(?![가-힣\d])\s?\)?|TAA\(회\)|SCSB|[a-zA-Z\-\d_]{2,}_[a-zA-Z\-\d_]{4,}|\(?\s?[a-zA-Z][\d]{9,}\s?\)?|\(?\s?[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}-[a-zA-Z\d]{2,}\s?\)?|\(?\s?[a-zA-Z0-9]{4,}-[a-zA-Z0-9]{4,}\s?\)?|\(?\s?\d{10,}\s?\)?|(?<![가-힣\d])[13-9]\d{5,}(?!가-힣\d])|(?<![가-힣\d])\d{3,4}(?![가-힣\d])|(?<![가-힣\d])\d{6}[\-_]\d{6}(?!가-힣\d])|(-)?\d{1,3}(,\d{3})+(\.\d+)?")
#p_rm = re.compile(r"(?<![가-힣\d])[13-9]\d{5,}(?!가-힣\d])|(?<![가-힣\d])\d{6}[\-_]\d{6}(?!가-힣\d])")
#                                 언더바로 연결된 영어,숫자 키(C1_3284792)  한자리 숫자(괄호,공백 포함)                                     2개의 하이픈으로 연결된 영어 및 숫자          하이픈 또는 언더바로 연결된 4자리 이상의 영숫자(주민번호 포함)  10자리키               날짜 아닐 6자리 이상 숫자      3~4자리의 단독 숫자
p_sign = re.compile("[^a-zA-Zㄱ-ㅎ가-힣0-9\s_()]|\(\)|_ | _|  ")
for f in files :
    n=f
    if p_rm.search(n[8:]):
        #print("있어")
        n= n[:8] + p_rm.sub(" ", n[8:])
        #print(n)
    else : print("없어", n)
    print(f, ">>>", p_sign.sub("", n))

20415347 3073 정상길 601,001,674 800206-1101016 원인서류 1차.pdf >>> 20415347 정상길 원인서류 1차pdf
20414261 1485 장행비 2,101,000.041_550703-2010518 원인서류 2차.pdf >>> 20414261 장행비   원인서류 2차pdf
20414336 1631 정효임 2,101.000389 790917-2114514 원인서류_3.pdf >>> 20414336 정효임  000389 원인서류pdf
20414507 1874 박성관 2101001090 521010-1841917 원인서류_4.pdf >>> 20414507 박성관원인서류pdf


In [57]:
from datetime import timedelta


day_e = datetime.today()
day_s = day_e + timedelta(days=31)
day_e_str = day_e.strftime("%Y%m%d")
day_s_str = day_s.strftime("%Y%m%d")
print(day_s_str, day_e_str)


TypeError: strptime() takes exactly 2 arguments (1 given)

In [113]:
for f in tqdm(os.listdir(r"D:\nas다운로드\upload\원인서류\nodocu")) :
    n = os.path.splitext(f)[0]
    if len(re.findall("-", n[8:]))==1 and re.search(r"[a-zA-Z]", n) :
        print(f)


  0%|          | 0/22956 [00:00<?, ?it/s]

20422648 S36BW-822031708140.pdf
20423013 S36BW-822022409510.pdf
20423083 김정웅_217LO034257-0001.pdf
20423233 지한민(일출해운)_216LO093093-0001.pdf
20423234 지한민(일출해운)_217LO012763-0001.pdf
20423266 원영준_218LO029303-0001.pdf
20423379 임창주(보람청과)_216LO168310-0001.pdf
20428531 에스비아이저축-KS.pdf
20442931 S36BW-822030420330.pdf
20444947 S36BW-822040811422.pdf
20499256 218RE013970-0001_손국승.pdf


In [109]:
a = "a-a-a"
len(re.findall("-", a))

2